In [1]:
%load_ext autoreload
%autoreload 2

# Curve Derivatives

In [2]:
import ipywidgets
import numpy
import pandas
import pathlib
import matplotlib.cm
from matplotlib import pyplot
import scipy

import bletl
import bletl_analysis

### Parse the raw data file

In [3]:
filepath = pathlib.Path(r'..\bletl\tests\data\BL1\NT_1200rpm_30C_DO-GFP75-pH-BS10_12min_20171221_121339.csv')
bldata = bletl.parse(filepath, lot_number=1515, temp=30)

### Plot the data of interest

In [6]:
def plot_well(well:str, filterset:str, last_cycle:int=None):
    x, y = bldata[filterset].get_timeseries(well)
    x = x[:last_cycle]
    y = y[:last_cycle]
    
    spline = bletl_analysis._crossvalidate_smoothing_spline(x, y, method='ucss', bounds=(0,1.2), k_folds=10)
    derivative = spline.derivative(1)
    
    # plotting
    fig, (up, down) = pyplot.subplots(dpi=140, nrows=2, sharex='col')
    up.scatter(x, y, s=.4, label='data')
    up.plot(x, spline(x), label='spline', linewidth=0.5)
    
    down.axhline(0, color='gray', linewidth=1, linestyle=':')
    d = numpy.linspace(0, x[-1], 1000)
    down.plot(d, derivative(d), label='1st derivative')
    
    # formatting
    #down.set_xlim(0, 23)
    down.set_xlabel('time   [h]')
    down.set_ylabel(f'd{filterset}/dt')
    up.set_ylabel(filterset)
    up.legend()
    
    pyplot.show()    
    return

ipywidgets.interact(
    plot_well,
    well=list(bldata['pH'].time.columns),
    filterset=list(bldata.keys()),
    last_cycle=range(10, len(bldata['pH'].time)+1)
);

interactive(children=(Dropdown(description='well', options=('A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', '…